## <small>
Copyright (c) 2017-21 Andrew Glassner

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
</small>



# Deep Learning: A Visual Approach
## by Andrew Glassner, https://glassner.com
### Order: https://nostarch.com/deep-learning-visual-approach
### GitHub: https://github.com/blueberrymusic
------

### What's in this notebook

This notebook is provided as a “behind-the-scenes” look at code used to make some of the figures in this chapter. It is cleaned up a bit from the original code that I hacked together, and is only lightly commented. I wrote the code to be easy to interpret and understand, even for those who are new to Python. I tried never to be clever or even more efficient at the cost of being harder to understand. The code is in Python3, using the versions of libraries as of April 2021. 

This notebook may contain additional code to create models and images not in the book. That material is included here to demonstrate additional techniques.

Note that I've included the output cells in this saved notebook, but Jupyter doesn't save the variables or data that were used to generate them. To recreate any cell's output, evaluate all the cells from the start up to that cell. A convenient way to experiment is to first choose "Restart & Run All" from the Kernel menu, so that everything's been defined and is up to date. Then you can experiment using the variables, data, functions, and other stuff defined in this notebook.

## Chapter 19: RNNs - Notebook 2: Characters

Some code here is inspired by https://github.com/karpathy/char-rnn

The Holmes data can be found at Project Gutenberg
https://www.gutenberg.org/ebooks/search/?query=holmes

I combined three books of short stories into one big text file:

- “The Adventures of Sherlock Holmes by Arthur Conan Doyle”
- “The Return of Sherlock Holmes by Arthur Conan Doyle”
- “The Memoirs of Sherlock Holmes by Arthur Conan Doyle”

Note that due to the use of random numbers, your output will almost surely be different than mine.

In [1]:
from keras.api.models import Sequential
from keras.api.layers import Dense, Activation
from keras.api.layers import LSTM
from keras.api.optimizers import RMSprop
import numpy as np
import random
import sys

In [2]:
# Workaround for Keras issues on Mac computers (you can comment this
# out if you're not on a Mac, or not having problems)
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
# Make a File_Helper for saving and loading files.

save_files = False

import os, sys, inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.insert(0, os.path.dirname(current_dir)) # path to parent dir
from DLBasics_Utilities import File_Helper
file_helper = File_Helper(save_files)

In [4]:
def get_text(input_file):
    # open the input file and do minor processing
    file = open(input_file, 'r', encoding='utf8') 
    text = file.read()
    file.close()
    #text = text.lower()
    # replace newlines with blanks, and double blanks with singles
    text = text.replace('\n',' ') 
    text = text.replace('  ', ' ')
    print('corpus length:', len(text))
    return text

In [5]:
def build_dictionaries(text):
    unique_chars = sorted(list(set(text)))
    print('total unique chars:', len(unique_chars))
    char_to_index = dict((ch, index) for index, ch in enumerate(unique_chars))
    index_to_char = dict((index, ch) for index, ch in enumerate(unique_chars))
    return (unique_chars, char_to_index, index_to_char)

In [6]:
def build_fragments(text, window_length):
    # make overlapping fragments of window_length characters
    fragments = []
    targets = []
    for i in range(0, len(text)-window_length, window_step):
        fragments.append(text[i: i + window_length])
        targets.append(text[i + window_length])
    print('number of fragments of length window_length=',window_length,':', len(fragments))
    return (fragments, targets)

In [7]:
def encode_training_data(fragments, window_length, targets, char_to_index, index_to_char):
    # Turn inputs and targets into one-hot versions
    X = np.zeros((len(fragments), window_length, len(char_to_index)), dtype=bool)
    y = np.zeros((len(fragments), len(char_to_index)), dtype=bool)
    for i, fragment in enumerate(fragments):
        for t, char in enumerate(fragment):
            X[i, t, char_to_index[char]] = 1
        y[i, char_to_index[targets[i]]] = 1
    return (X, y)

In [8]:
def build_model(window_length, num_unique_chars):
    # build the model. Two layers of a single LSTM cell with 128 elements of memory,
    # then a dense layer with as many outputs as there are characters (89)
    # We'll train with the RMSprop optimizer. Some experiments suggest that
    # a learning rate of 0.01 is a good place to start.
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(window_length, num_unique_chars)))
    model.add(LSTM(128))
    model.add(Dense(num_unique_chars, activation='softmax'))
    optimizer = RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    return model

In [9]:
# adjust our probabilities to add "heat"
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [10]:
# print a string to the screen and also save it in the file
def print_string(out_str='', file_writer=None):
    print(out_str, end='')
    if file_writer != None:
        file_writer.write(out_str)

In [11]:
def generate_text(model, X, y, number_of_epochs, temperatures, index_to_char, char_to_index, file_writer):
    # train the model, output generated text after each iteration
    for iteration in range(number_of_epochs):
        print_string('--------------------------------------------------\n', file_writer)
        print_string('Iteration '+str(iteration)+'\n', file_writer)
        history = model.fit(X, y, batch_size=batch_size, epochs=1)
        start_index = random.randint(0, len(text) - window_length - 1)

        for temperature in temperatures:
            print_string('\n----- temperature: '+str(temperature)+'\n', file_writer)
            sentence = text[start_index: start_index + window_length]
            generated = sentence
            print_string('----- Generating with seed: <'+sentence+'>\n', file_writer)

            for i in range(generated_text_length):
                x = np.zeros((1, window_length, len(index_to_char)))
                for t, char in enumerate(sentence):
                    x[0, t, char_to_index[char]] = 1.

                preds = model.predict(x, verbose=0)[0]
                next_index = sample(preds, temperature)
                next_char = index_to_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

            print_string(generated+'\n\n', file_writer)
            file_writer.flush()
    #print_string('\n', file_writer)

In [12]:
# set the globals
window_length = 40
window_step = 3
number_of_epochs = 100
generated_text_length = 1000
batch_size = 100
input_dir = file_helper.get_input_data_dir()
output_dir = file_helper.get_saved_output_dir()
file_helper.check_for_directory(output_dir)

input_file = input_dir+'/holmes.txt'
output_file =  output_dir+'/holmes-by-char.txt'
File_writer = open(output_file, 'w')

In [13]:
# get text data structures, build the model

text = get_text(input_file)
unique_chars, char_to_index, index_to_char = build_dictionaries(text)
fragments, targets = build_fragments(text, window_length)
X, y = encode_training_data(fragments, window_length, targets, char_to_index, index_to_char)
model = build_model(window_length, len(char_to_index))
# Show the model we're using
model.summary()

corpus length: 1637265
total unique chars: 89
number of fragments of length window_length= 40 : 545742


D:\CourseIAGenAI\git\iagenai\fmi-2024-02-ann-dnn\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Argument(s) not recognized: {'lr': 0.01}

In [14]:
number_of_epochs = 2
temperatures = [0.5, 1.0, 1.5]
generate_text(model, X, y, number_of_epochs, temperatures, index_to_char, char_to_index, File_writer)
# wrap up when we're done
File_writer.close()

--------------------------------------------------
Iteration 0
Epoch 1/1
545742/545742 [==============================] - 1887s 3ms/step - loss: 1.7401

----- temperature: 0.5
----- Generating with seed: <wax. It would be done so quickly and so >
wax. It would be done so quickly and so sure that the crough to be the canstable before such was farst of his sine of my remarked at the stood, which was your door the lack which was the window. She shall do the distle exaction was some find the congint upon the door, but the one of the ended and to her that I do not the continully which I had on the story which he must had let interest upon the rather was that the tables was the strength to me to his going of the dear brige with the hand with the morning of grimpable better his the maged of the look was for the ling and what had the place, and the door which had sure to her he open the look and in the most upon the plook of the rameself which have been in that seemed to his hear that, and she

/Users/Andrew/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


wax. It would be done so quickly and so certain the Ig as in by u hear, marked not being foubky in exremed he were for any front of much excellent to light, with her cir spomer, so by a fipouping came in max. Exmuray non? “Theistlefticaltvess the repleater, and I was the instiingcud eldest faken that I were imestors.  I saw you, he was  out andrises to seemed parttang a greated at ankers fralifias difficult fouttly tork. Have a sfied and hard. As. saw the treclipe, were right that I was all one until sittle Abuttle in it was very front and provained after the stepn of every the Mance twerece. I do , and we shall anfeughed have to interest and in nour, the fist cais. Let will be one intereming disperfatice, which he rement to take ancorder in a man, it is which had been unterent, and so your house, herepyed. ’t you toot. Then then he do some begang. She stange, we had proding thane wife stood measfeded, out I moodd, and the druss." I amselfro tin rough trescratable has been an his Wriss